### import

In [1]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
import datetime

### function

In [2]:
주민조건 = [2,3,4,5,6,7,8,9,2,3,4,5]
법인조건 = [1,2,1,2,1,2,1,2,1,2,1,2]
사업자조건 = [1,3,7,1,3,7,1,3,5]
# 주민등록번호 유효성 검사 > 통과시 개인. 개인인데 주민번호 오류인것, 법인등록번호가 유효성 통과한 경우 확인 필요
############################# 사업자 검사 추가, 14자리이나 000으로 시작하는 경우, 000 제거하고 사업자 등록번호 검사,통과하면 법인, 통과하지 못하면 개인.
############################# 셋다 통과하지 못하거나, 10자리인데 사업자 번호 통과하지 못하면 유효하지 않은 번호로 따로 열 만들어서 적어주자. 
def only_num(regit_num:str) :
    return re.sub("[^0-9]", "", regit_num)


def 식별번호유효성검사 (regit_num:str, cond_list:list, minus_num:int, mod_num:int) :
    sum = 0
    for i in range(len(regit_num)-1) :
        sum += int(regit_num[i]) * cond_list[i]
    if len(cond_list) == 9 : # 사업자번호인경우
        sum += int(regit_num[8]) * cond_list[8] // 10
    return int(regit_num[-1]) == (minus_num - (sum % mod_num)) % 10 # 일의자리만 얻기 위해 % 10


def 주민번호유효성검사(regit_num:str) :
    regit_num = only_num(regit_num)
    if len(regit_num) == 13 :
        month = int(regit_num[2:4])
        day = int(regit_num[4:6])

        # 월 일 유효성 검증
        if month > 12 or day > 31 : return False
        elif month in [4,6,9,11] : 
            if day > 30 : return False
        elif month == 2 :
            if day > 29 : return False

        # 주민번호 유효성 검증
        # 내국인
        number = 0
    
        # 내국인
        if regit_num[6] in ["1","2","3","4","9","0"] : 
            return 식별번호유효성검사(regit_num, 주민조건, minus_num=11, mod_num=11)
        # 외국인 
        else : 
            return 식별번호유효성검사(regit_num, 주민조건, minus_num=13, mod_num=11)
    else :
        return False

# 법인등록번호 유효성 체크, 인터넷등기소 > 법입 > 열람
def 법인번호유효성검사(regit_num:str) : # 0으로시작하는 관서번호는 없음
    regit_num = only_num(regit_num)
    if len(regit_num) != 13 : return False
    else :
        # 등기관서번호
        등기관서번호 = (1101,1103,1201,1211,1244,1245,1311,1312,1313,1314,1341,1342,1343,1344,1345,1346,1348,1349,1354,1355,
        1356,1357,1358,1359,1401,1411,1412,1413,1414,1441,1442,1443,1444,1445,1446,1447,1448,1449,1450,1451,1452,1501,1511,1512,
        1513,1541,1542,1543,1544,1545,1546,1601,1611,1612,1613,1614,1615,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,
        1652,1701,1711,1712,1713,1714,1715,1716,1717,1718,1743,1744,1745,1747,1748,1749,1750,1751,1752,1754,1755,1756,1757,1758,
        1759,1760,1801,1811,1841,1843,1844,1847,1849,1851,1911,1912,1913,1914,1915,1941,1942,1943,1944,1945,1946,1947,1948,1949,
        1950,1951,1952,1953,1954,1955,2001,2011,2012,2013,2014,2043,2044,2045,2046,2047,2049,2050,2052,2053,2054,2055,2056,2057,
        2058,2059,2060,2061,2062,2101,2111,2112,2113,2141,2142,2143,2144,2145,2146,2147,2148,2149,2201,2241,2301,2341,2401,2501,
        2601,2701,2802,2841,2842,2843,2844,2845,2846,2847,2849,2850)
        법인종류번호 = (11,12,13,14,15,21,22,31,32,33,34,35,36,38,39,40,41,43,44,46,47,48,49,51,52,53,54,55,61,71,81,82,83,84,85,86)
        if int(regit_num[:4]) not in 등기관서번호 : 
            return False
        # 법인종류번호
        elif int(regit_num[4:6]) not in 법인종류번호 : 
            return False
        else :
            # 유효성 검증
            return 식별번호유효성검사(regit_num, 법인조건, minus_num=10, mod_num=10)
            
# 홈텍스 > 조회/발급 > 사업자상태 > 등록번호/주민번호 조회 or bizno.net
def 사업자번호유효성검사 (regit_num : str) :
    regit_num = only_num(regit_num)
    if len(regit_num)==13 and (re.match('000',regit_num)) : 
        regit_num = regit_num[3:]
    elif len(regit_num)==13 and (re.match('000',regit_num)==None) :
        return False
    elif len(regit_num) != 10 : return False
    else : pass

    # 유효성 검사
    return 식별번호유효성검사(regit_num, 사업자조건, minus_num=10, mod_num=10)


# 주민번호로 성별 체크
def 성별분류(regit_num:str, character:str) :
    if character == "법인" : return "법인"
    else :
        # 홀수면
        if int(regit_num[8]) % 2 == 1 : return "남자"
        else : return "여자"

### 채무자조회새창 파일 읽기

In [7]:
###########################################
wd = r"D:\3.자산\프로젝트\2023 전산데이터 클렌징 및 업무개편 프로젝트\check\개인법인,성별"
file_name = "채무자조회새창_20230810_1106" + ".pkl"
########################################### ######
df = pd.read_pickle(join(wd, file_name))

### 개인법인 체크

In [8]:
# 주민번호 유효성 검사 결과
# 주민번호유효성결과 열 추가
result1 = pd.Series(df.주민번호인.apply(주민번호유효성검사))
df = pd.concat([df, result1.rename("주민번호유효성결과")], axis=1)

result2 = df.주민번호인.apply(법인번호유효성검사)
df = pd.concat([df, result2.rename("법인번호유효성결과")], axis=1)

result3 = df.주민번호인.apply(사업자번호유효성검사)
df = pd.concat([df, result3.rename("사업자번호유효성결과")], axis=1)

In [9]:
# 새로운 개인법인 작성
x = df["주민번호유효성결과"]
y = df["법인번호유효성결과"]
z = df["사업자번호유효성결과"]
# 개인, 법인, 오류 여부를 나타내는 조건 평가
cond1 = x & ~y & ~z
cond2 = (~x & y & ~z) | (~x & ~y & z)

# 조건을 기반으로 값을 설정
new_values = np.select([cond1, cond2], ["개인", "법인"], default="오류")
df = pd.concat([df, pd.DataFrame({"new개인법인": new_values})], axis=1)

# 유효성 검사 결과가 기존 전산값과 다른 것만 엑셀파일로
cols_for_save = ["채무자키", "성명", "주민번호인", "개인법인", "new개인법인", "주민번호유효성결과", "법인번호유효성결과", "사업자번호유효성결과"]
df[df["new개인법인"] != df["개인법인"]][cols_for_save].to_excel(join(wd,"check개인법인.xlsx"), index=False)

### 성별 체크

In [ ]:
df["성별검사결과"] = df.주민번호인.apply(성별분류)
df[df["성별검사결과"] != df["성별"]][["채무자키", "성명", "주민번호인", "개인법인", "성별", "성별검사결과"]].to_excel(join(wd, "check성별.xlsx"), index=False)

### 국세청 사업자 휴폐업 조회

In [12]:
import requests

HOMETAX_URL = 'https://teht.hometax.go.kr/wqAction.do?actionId=ATTABZAA001R08&screenId=UTEABAAA13&popupYn=false&realScreenId='

def checkTaxStatus (사업자등록번호):

	#remove '-' from bizId
	removed = bizId.replace ('-','')

	#make XML
	xml = '<map id=""><pubcUserNo/><mobYn>N</mobYn><inqrTrgtClCd>1</inqrTrgtClCd><txprDscmNo>'\
	+ removed +'</txprDscmNo><dongCode></dongCode><psbSearch></psbSearch><map id="userReqInfoVO"/></map>'
	headers = {'Content-Type': 'application/xml'}
	
	r = requests.post (HOMETAX_URL, data=xml, headers=headers).text

	begin_tag = '<trtCntn>'
	end_tag = '</trtCntn>'
	begin_idx = r.find (begin_tag)
	end_idx = r.find(end_tag)

	result = r[begin_idx+9: end_idx]
	print (result)

	return result

In [13]:
사업자등록번호 = "308-81-04817"
checkTaxStatus(사업자등록번호)

폐업자 (부가가치세 일반과세자, 폐업일자:1998-10-20) 입니다.


'폐업자 (부가가치세 일반과세자, 폐업일자:1998-10-20) 입니다.'

### 통신사업판매자(대부분의 사업체?) 조회 - 사업자등록번호/업체명

In [19]:
# 통합본 파일 읽기 (모든 칼럼 str)
df = pd.read_pickle(r"D:\3.자산\프로젝트\2023 전산데이터 클렌징 및 업무개편 프로젝트\check\개인법인,성별\통신판매사업자_공정거래위원회_230810\통신판매사업자_ALL_전지역통합_230810.pkl")
df.head(1)

,통신판매번호,신고기관명,상호,사업자등록번호,법인여부,대표자명,전화번호,전자우편,신고일자,사업장소재지,사업장소재지(도로명),업소상태,신고기관 대표연락처,판매방식,취급품목,인터넷도메인,호스트서버소재지
0,2004-01914,서울특별시 종로구,지성의료기,,개인,김정구,02 2213 1367,webmast**@chimdo/com,,서울특별시 종로구 종로**가********,서울특별시 종로구 대학로**길 ** (종*****,통신판매업 신고,02-2148-2294,,,www.chimdo.com,양천구신정동973-35


In [21]:
df[df["상호"].str.contains("연산레미콘")]
# 결측치(float) 있을 때
# df[df["상호"].notnull() &  df["상호"].str.contains("레미콘")]

,통신판매번호,신고기관명,상호,사업자등록번호,법인여부,대표자명,전화번호,전자우편,신고일자,사업장소재지,사업장소재지(도로명),업소상태,신고기관 대표연락처,판매방식,취급품목,인터넷도메인,호스트서버소재지


In [11]:
# 상호검색
# fillna("")를 한 파일인데도 결측치가 있네..(결측치는 float라 str함수에는 오류남)
df[df["상호"].notnull() &  df["상호"].str.contains("레미콘")]

,통신판매번호,신고기관명,상호,사업자등록번호,법인여부,대표자명,전화번호,전자우편,신고일자,사업장소재지,사업장소재지(도로명),업소상태,신고기관 대표연락처,판매방식,취급품목,인터넷도메인,호스트서버소재지
196077,2011-서울강북-0138,서울특별시 강북구,평화레미콘,210-48-35433,개인,김완용,,,2011-03-23 00:00:00,,,통신판매업 폐업,02-901-6454,인터넷,의류/패션/잡화/뷰티/ 기타,옥션 지마켓,
84678,2022-인천계양-1394,인천광역시 계양구,(주)레아상사레미콘,501-81-26818,법인,이소영,,,2022-12-27 00:00:00,,,통신판매업 폐업,032-450-5486,인터넷,기타,rhearemicon.modoo.at,
261239,2019-고양덕양구-0839,경기도 고양시 덕양구,우진레미콘주식회사,128-86-91517,법인,박종공,031-971-8980,woojinr**@daum.net,2019-06-27 00:00:00,경기도 고양시 덕양구 도내동 ******번*****,경기도 고양시 덕양구 흥도로 ****** **********외 **필지 *****,통신판매업 신고,031-8075-5454,인터넷,종합몰/ 가구/수납용품,this-isnt.com,경기도 성남시 분당구 성남대로925번길 36/ KT (야탑동)
42142,2008-충북청주-0450,충청북도 청주시,전국지방화물 삼성레미콘 이삿짐센터 창진모터스,314-06-22910,개인,황의문,,,2008-10-07 00:00:00,,,직권말소,043-201-1045,인터넷/ 기타,기타,www.2434.co.kr,
50359,2021-경북칠곡-0269,경상북도 칠곡군,(주) 백산레미콘,504-81-27868,법인,심덕룡/이용주,054-983-0809,tolvi**@gmail.com,2021-07-12 00:00:00,경상북도 칠곡군 가산면 학상리 **********,경상북도 칠곡군 가산면 학상**길*****,통신판매업 신고,054-979-6524,인터넷,레저/여행/공연,803applehotel.com,서울특별시 성동구 뚝섬로1길 31/ 404호~405호 (성수동1가)
6685,2020-제주건입-0020,제주특별자치도 제주시,(주)미륭레미콘 제주지점(심스호텔),617-81-05724,법인,이태희,,,2020-07-21 00:00:00,,,통신판매업 폐업,064-120,인터넷,레저/여행/공연,www.jejusims.com,
